# Bedrock Agent with Knowledge Base
Create an agent that can query a knowledge base using RAG

In [ ]:
import boto3
import json
import time

bedrock_agent = boto3.client('bedrock-agent', region_name='us-east-1')
s3 = boto3.client('s3')
iam = boto3.client('iam')

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
KB_NAME = 'product-docs-kb'
AGENT_NAME = 'product-assistant'

## 1. Create S3 Bucket and Upload Documents

In [ ]:
BUCKET_NAME = f"{KB_NAME}-{ACCOUNT_ID}"

try:
    s3.create_bucket(Bucket=BUCKET_NAME)
    print(f"✓ Created bucket: {BUCKET_NAME}")
except: 
    print(f"✓ Bucket exists: {BUCKET_NAME}")

# Upload sample documents
docs = [
    ("product1.txt", "Product X is a cloud storage solution with 99.99% uptime. Price: $10/month."),
    ("product2.txt", "Product Y is an analytics platform with real-time dashboards. Price: $50/month.")
]

for filename, content in docs:
    s3.put_object(Bucket=BUCKET_NAME, Key=filename, Body=content)
    print(f"✓ Uploaded {filename}")

## 2. Create IAM Role for Knowledge Base

In [ ]:
kb_role_name = f"{KB_NAME}-role"

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "bedrock.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

kb_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["s3:GetObject", "s3:ListBucket"], "Resource": [f"arn:aws:s3:::{BUCKET_NAME}/*", f"arn:aws:s3:::{BUCKET_NAME}"]},
        {"Effect": "Allow", "Action": "bedrock:InvokeModel", "Resource": "*"},
        {"Effect": "Allow", "Action": "aoss:APIAccessAll", "Resource": "*"}
    ]
}

try:
    response = iam.create_role(RoleName=kb_role_name, AssumeRolePolicyDocument=json.dumps(trust_policy))
    KB_ROLE_ARN = response['Role']['Arn']
    iam.put_role_policy(RoleName=kb_role_name, PolicyName='kb-policy', PolicyDocument=json.dumps(kb_policy))
    time.sleep(10)
    print(f"✓ Created KB role")
except iam.exceptions.EntityAlreadyExistsException:
    KB_ROLE_ARN = iam.get_role(RoleName=kb_role_name)['Role']['Arn']
    print(f"✓ KB role exists")

## 3. Create Knowledge Base

In [ ]:
response = bedrock_agent.create_knowledge_base(
    name=KB_NAME,
    roleArn=KB_ROLE_ARN,
    knowledgeBaseConfiguration={
        'type': 'VECTOR',
        'vectorKnowledgeBaseConfiguration': {
            'embeddingModelArn': f'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'
        }
    },
    storageConfiguration={
        'type': 'OPENSEARCH_SERVERLESS',
        'opensearchServerlessConfiguration': {
            'collectionArn': 'arn:aws:aoss:us-east-1:' + ACCOUNT_ID + ':collection/kb-collection',
            'vectorIndexName': 'bedrock-kb-index',
            'fieldMapping': {
                'vectorField': 'vector',
                'textField': 'text',
                'metadataField': 'metadata'
            }
        }
    }
)

KB_ID = response['knowledgeBase']['knowledgeBaseId']
print(f"✓ Created knowledge base: {KB_ID}")

## 4. Create Data Source

In [ ]:
response = bedrock_agent.create_data_source(
    knowledgeBaseId=KB_ID,
    name='s3-docs',
    dataSourceConfiguration={
        'type': 'S3',
        's3Configuration': {
            'bucketArn': f'arn:aws:s3:::{BUCKET_NAME}'
        }
    }
)

DATA_SOURCE_ID = response['dataSource']['dataSourceId']
print(f"✓ Created data source: {DATA_SOURCE_ID}")

## 5. Ingest Documents

In [ ]:
response = bedrock_agent.start_ingestion_job(
    knowledgeBaseId=KB_ID,
    dataSourceId=DATA_SOURCE_ID
)

job_id = response['ingestionJob']['ingestionJobId']
print(f"Started ingestion job: {job_id}")

# Wait for completion
while True:
    status = bedrock_agent.get_ingestion_job(
        knowledgeBaseId=KB_ID,
        dataSourceId=DATA_SOURCE_ID,
        ingestionJobId=job_id
    )['ingestionJob']['status']
    
    if status == 'COMPLETE':
        print("✓ Ingestion complete")
        break
    elif status == 'FAILED':
        print("✗ Ingestion failed")
        break
    
    print(f"Status: {status}")
    time.sleep(10)

## 6. Create Agent with Knowledge Base

In [ ]:
# Create agent role
agent_role_name = f"{AGENT_NAME}-role"
agent_trust = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "bedrock.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}

try:
    response = iam.create_role(RoleName=agent_role_name, AssumeRolePolicyDocument=json.dumps(agent_trust))
    AGENT_ROLE_ARN = response['Role']['Arn']
    iam.attach_role_policy(RoleName=agent_role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess')
    time.sleep(10)
except iam.exceptions.EntityAlreadyExistsException:
    AGENT_ROLE_ARN = iam.get_role(RoleName=agent_role_name)['Role']['Arn']

# Create agent
response = bedrock_agent.create_agent(
    agentName=AGENT_NAME,
    agentResourceRoleArn=AGENT_ROLE_ARN,
    foundationModel='us.anthropic.claude-3-5-sonnet-20241022-v2:0',
    instruction="You are a product assistant. Answer questions about products using the knowledge base."
)

AGENT_ID = response['agent']['agentId']
print(f"✓ Created agent: {AGENT_ID}")

## 7. Associate Knowledge Base with Agent

In [ ]:
response = bedrock_agent.associate_agent_knowledge_base(
    agentId=AGENT_ID,
    agentVersion='DRAFT',
    knowledgeBaseId=KB_ID,
    description='Product documentation',
    knowledgeBaseState='ENABLED'
)

print("✓ Associated knowledge base with agent")

## 8. Prepare and Test Agent

In [ ]:
bedrock_agent.prepare_agent(agentId=AGENT_ID)

while True:
    status = bedrock_agent.get_agent(agentId=AGENT_ID)['agent']['agentStatus']
    if status == 'PREPARED':
        break
    time.sleep(5)

# Create alias
response = bedrock_agent.create_agent_alias(agentId=AGENT_ID, agentAliasName='prod')
AGENT_ALIAS_ID = response['agentAlias']['agentAliasId']

print("✓ Agent ready")

In [ ]:
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

def query_agent(prompt):
    response = bedrock_agent_runtime.invoke_agent(
        agentId=AGENT_ID,
        agentAliasId=AGENT_ALIAS_ID,
        sessionId='kb-session',
        inputText=prompt
    )
    
    result = ""
    for event in response['completion']:
        if 'chunk' in event:
            result += event['chunk']['bytes'].decode('utf-8')
    return result

print(query_agent("What products do you have and what are their prices?"))